In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import metrics
from sklearn.model_selection import train_test_split

In [48]:
data = pd.read_csv('/content/drive/My Drive/2022-1/기계학습/master_3_1.csv', index_col=0, encoding='cp949')

In [2]:
train = pd.read_csv('../dataset/train_D_U_O.csv').iloc[:,1:]
test = pd.read_csv('../dataset/test_D_U_O.csv').iloc[:,1:]
# X = train.drop(['outcome_critical'], axis=1)
# y = train['outcome_critical']
# X_test = test.drop(['outcome_critical'], axis=1)
# y_test = test['outcome_critical']
# # Split dataset into training set and test set
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2022) # 80% training and 20% test

In [6]:
train.columns

Index(['age', 'chiefcom_abdominal_pain', 'chiefcom_chest_pain',
       'chiefcom_fever_chills', 'ed_dbp_last', 'ed_heartrate_last',
       'ed_los_hours', 'ed_pain_last', 'ed_resprate_last', 'ed_sbp_last',
       'gender', 'n_hosp_365d', 'n_icu_365d', 'n_med', 'n_medrecon',
       'outcome_hospitalization', 'outcome_inhospital_mortality', 'score_CCI',
       'stay_id', 'triage_acuity', 'triage_resprate', 'triage_sbp',
       'outcome_critical'],
      dtype='object')

In [7]:
print(data.isna().sum())
print(data.shape)
print(list(data.columns))

subject_id         0
stay_id            0
gender             0
in_year            0
age                0
                  ..
ed_dbp_last        0
ed_pain_last       0
n_med              0
n_medrecon         0
critical_result    0
Length: 79, dtype: int64
(448972, 79)
['subject_id', 'stay_id', 'gender', 'in_year', 'age', 'outcome_inhospital_mortality', 'ed_death', 'before_ed_mortality', 'outcome_hospitalization', 'n_ed_30d', 'n_ed_90d', 'n_ed_365d', 'outcome_ed_revisit_3d', 'n_hosp_30d', 'n_hosp_90d', 'n_hosp_365d', 'n_icu_30d', 'n_icu_90d', 'n_icu_365d', 'ed_los_hours', 'triage_temperature', 'triage_heartrate', 'triage_resprate', 'triage_o2sat', 'triage_sbp', 'triage_dbp', 'triage_pain', 'triage_acuity', 'chiefcom_chest_pain', 'chiefcom_abdominal_pain', 'chiefcom_headache', 'chiefcom_shortness_of_breath', 'chiefcom_back_pain', 'chiefcom_cough', 'chiefcom_nausea_vomiting', 'chiefcom_fever_chills', 'chiefcom_syncope', 'chiefcom_dizziness', 'eci_CHF', 'eci_Arrhythmia', 'eci_Valvular', 'e

In [49]:
data['death_result'] = data['ed_death'] 

In [50]:
data=data.drop('ed_death',axis=1)
data

,subject_id,stay_id,gender,in_year,age,outcome_inhospital_mortality,before_ed_mortality,outcome_hospitalization,n_ed_30d,n_ed_90d,...,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,critical_result,death_result
index,,,,,,,,,,,,,,,,,,,,,
335314,10000032,33258284,0,2180,52,False,False,True,0,0,...,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0,False,False
335315,10000032,38112554,0,2180,52,False,False,True,0,1,...,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0,False,False
335316,10000032,32952584,0,2180,52,False,False,True,1,2,...,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0,True,False
335317,10000032,39399961,0,2180,52,False,False,True,2,3,...,96.0,18.0,97.0,86.0,45.0,0.0,8.0,14.0,True,False
335318,10000032,35968195,0,2180,52,False,False,True,2,3,...,91.0,18.0,99.0,98.0,60.0,0.0,6.0,7.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167406,19999784,34149746,1,2119,57,False,False,True,0,2,...,72.0,16.0,100.0,131.0,92.0,0.0,1.0,8.0,False,False
167407,19999828,30712109,0,2147,46,False,False,True,0,0,...,78.0,17.0,99.0,108.0,87.0,9.0,11.0,8.0,False,False
167408,19999828,32917002,0,2149,48,False,False,True,0,0,...,109.0,15.0,96.0,111.0,78.0,4.0,8.0,11.0,True,False


In [3]:
# x y 로 데이터 분리 후 dmatrix로 변환 (y1=치명적인지 아닌지 y2= 부상정도 )
data_x = train.drop(['outcome_critical', 'score_CCI'], axis=1)
data_y = pd.concat([train['outcome_critical'], train['score_CCI']], axis=1)
data_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84270 entries, 0 to 84269
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           84270 non-null  int64  
 1   chiefcom_abdominal_pain       84270 non-null  bool   
 2   chiefcom_chest_pain           84270 non-null  bool   
 3   chiefcom_fever_chills         84270 non-null  bool   
 4   ed_dbp_last                   84270 non-null  float64
 5   ed_heartrate_last             84270 non-null  float64
 6   ed_los_hours                  84270 non-null  float64
 7   ed_pain_last                  84270 non-null  float64
 8   ed_resprate_last              84270 non-null  float64
 9   ed_sbp_last                   84270 non-null  float64
 10  gender                        84270 non-null  int64  
 11  n_hosp_365d                   84270 non-null  int64  
 12  n_icu_365d                    84270 non-null  int64  
 13  n

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
# 레이블 데이터
input_labels = ['False','True']
encoder = LabelEncoder()
# 문자를 레이블로 변환

data_y['outcome_critical'] =  encoder.fit_transform(data_y['outcome_critical'].values)
# data['death_result'] =  encoder.fit_transform(data['death_result'].values)
# data['critical_result']

In [48]:

data_y.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84270 entries, 0 to 84269
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   outcome_critical  84270 non-null  int64
 1   score_CCI         84270 non-null  int64
dtypes: int64(2)
memory usage: 1.3 MB


In [7]:
data_x = data_x.astype(np.float32)
data_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84270 entries, 0 to 84269
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           84270 non-null  float32
 1   chiefcom_abdominal_pain       84270 non-null  float32
 2   chiefcom_chest_pain           84270 non-null  float32
 3   chiefcom_fever_chills         84270 non-null  float32
 4   ed_dbp_last                   84270 non-null  float32
 5   ed_heartrate_last             84270 non-null  float32
 6   ed_los_hours                  84270 non-null  float32
 7   ed_pain_last                  84270 non-null  float32
 8   ed_resprate_last              84270 non-null  float32
 9   ed_sbp_last                   84270 non-null  float32
 10  gender                        84270 non-null  float32
 11  n_hosp_365d                   84270 non-null  float32
 12  n_icu_365d                    84270 non-null  float32
 13  n

In [13]:
print(data_y.isna().sum())

outcome_critical    0
score_CCI           0
dtype: int64


In [14]:
data_y.head()

,outcome_critical,score_CCI
0,False,0
1,False,0
2,False,4
3,False,0
4,False,4


In [15]:
# epoch = [32, 64, 128]
# batch_sizes = [8, a16, 32]

In [91]:
train['score_CCI'].unique()

array([ 0,  4,  1,  2,  3, 14,  7, 11,  6,  5, 16, 10, 12, 15,  8, 13,  9,
       18, 17, 21, 19, 20, 22, 23, 24], dtype=int64)

In [92]:
train.columns

Index(['age', 'chiefcom_abdominal_pain', 'chiefcom_chest_pain',
       'chiefcom_fever_chills', 'ed_dbp_last', 'ed_heartrate_last',
       'ed_los_hours', 'ed_pain_last', 'ed_resprate_last', 'ed_sbp_last',
       'gender', 'n_hosp_365d', 'n_icu_365d', 'n_med', 'n_medrecon',
       'outcome_hospitalization', 'outcome_inhospital_mortality', 'score_CCI',
       'stay_id', 'triage_acuity', 'triage_resprate', 'triage_sbp',
       'outcome_critical', 'Set'],
      dtype='object')

In [14]:
train['Set'] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))
target = ['outcome_critical', 'score_CCI']
train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [25]:
encoder = LabelEncoder()
train['outcome_critical'] =  encoder.fit_transform(train['outcome_critical'].values)

In [28]:
features = [ col for col in train.columns if col not in ['Set']+target]
X_train = train[features].values[train_indices]
X_train = X_train.astype(np.float32)
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
X_valid = X_valid.astype(np.float32)
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
X_test = X_test.astype(np.float32)
y_test = train[target].values[test_indices]

## 1번째 모델

In [17]:
bc = keras.losses.BinaryCrossentropy()
ce = keras.losses.CategoricalCrossentropy()

In [18]:
def custom_loss(y_true, y_pred):
    y_class1_true = y_true[:, 0]
    y_class1_pred = y_pred[:, 0]
    
    y_class2_true = y_true[:, 1]
    y_class2_pred = y_pred[:, 1]
    
    class1_loss = bc(y_class1_true, y_class1_pred)
    class2_loss = ce(y_class2_true, y_class2_pred)
    
    return 0.5 * class1_loss + 0.5 * class2_loss

In [31]:
# model = Sequential()
input = keras.layers.Input(shape=(data_x.shape[1],), name='input')
x = Dense(32, activation='relu', kernel_initializer='normal', name='dense_1')(input)
x = Dropout(rate=0.2, name='dropout_1')(x)
x = Dense(64, input_shape=(data_x.shape[1],), activation='relu', kernel_initializer='normal', name='dense_2')(x)
x = Dense(32, activation='relu', kernel_initializer='normal', name='dense_3')(x)
branch1_class = Dense(1, activation='sigmoid', kernel_initializer='normal', name='branch1_class')(x)
branch2_class = Dense(1, activation='softmax', kernel_initializer='normal', name='branch2_class')(x)
out = keras.layers.concatenate([branch1_class, branch2_class], name='concat')
model = keras.models.Model(input, out)
model.compile(loss=custom_loss, optimizer='adam',  metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 21)]         0           []                               
                                                                                                  
 dense_1 (Dense)                (None, 32)           704         ['input[0][0]']                  
                                                                                                  
 dropout_1 (Dropout)            (None, 32)           0           ['dense_1[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 64)           2112        ['dropout_1[0][0]']              
                                                                                            

In [32]:
ckpt_path = 'mtl'
callbacks_list = [
    # accuracy 기준 가장 높은 모델의 weight 저장
    keras.callbacks.ModelCheckpoint(
        filepath = ckpt_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # EarlyStopping
    # tf.keras.callbacks.EarlyStopping(
    #    monitor='val_loss',
    #    mode='min',
    #    verbose=1, 
    #    patience=20
    # )
]

In [33]:
hist = model.fit(X_train, y_train, validation_data=[X_valid, y_valid], epochs=20)

Epoch 1/20
2108/2108 [==============================] - 25s 11ms/step - loss: 192.9767 - accuracy: 0.6963 - val_loss: 192.6385 - val_accuracy: 0.6983
Epoch 2/20
2108/2108 [==============================] - 23s 11ms/step - loss: 192.9719 - accuracy: 0.6969 - val_loss: 192.6385 - val_accuracy: 0.6983
Epoch 3/20
2108/2108 [==============================] - 22s 10ms/step - loss: 192.9848 - accuracy: 0.6969 - val_loss: 192.6385 - val_accuracy: 0.6983
Epoch 4/20
2108/2108 [==============================] - 23s 11ms/step - loss: 192.9738 - accuracy: 0.6969 - val_loss: 192.6385 - val_accuracy: 0.6983
Epoch 5/20
2108/2108 [==============================] - 23s 11ms/step - loss: 192.9725 - accuracy: 0.6969 - val_loss: 192.6385 - val_accuracy: 0.6983
Epoch 6/20
2108/2108 [==============================] - 22s 10ms/step - loss: 192.9785 - accuracy: 0.6969 - val_loss: 192.6385 - val_accuracy: 0.6983
Epoch 7/20
2108/2108 [==============================] - 23s 11ms/step - loss: 192.9791 - accuracy: 0

## 2번째 모델

In [3]:
train['Set'] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))
target = ['outcome_critical', 'ed_los_hours']
train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [6]:
encoder = LabelEncoder()
train['outcome_critical'] =  encoder.fit_transform(train['outcome_critical'].values)

In [7]:
features = [ col for col in train.columns if col not in ['Set']+target]
X_train = train[features].values[train_indices]
X_train = X_train.astype(np.float32)
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
X_valid = X_valid.astype(np.float32)
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
X_test = X_test.astype(np.float32)
y_test = train[target].values[test_indices]

In [8]:
mse = keras.losses.MeanSquaredError()
bc = keras.losses.BinaryCrossentropy()

In [9]:
def custom_loss2(y_true, y_pred):
    y_class1_true = y_true[:, 0]
    y_class1_pred = y_pred[:, 0]
    
    y_class2_true = y_true[:, 1]
    y_class2_pred = y_pred[:, 1]
    
    class1_loss = bc(y_class1_true, y_class1_pred)
    class2_loss = mse(y_class2_true, y_class2_pred)
    
    return 0.5 * class1_loss + 0.5 * class2_loss

In [11]:
# model = Sequential()
input = keras.layers.Input(shape=(X_train.shape[1],), name='input')
x = Dense(32, activation='relu', kernel_initializer='normal', name='dense_1')(input)
x = Dropout(rate=0.2, name='dropout_1')(x)
x = Dense(64, input_shape=(X_train.shape[1],), activation='relu', kernel_initializer='normal', name='dense_2')(x)
x = Dense(32, activation='relu', kernel_initializer='normal', name='dense_3')(x)
branch1_class = Dense(1, activation='sigmoid', kernel_initializer='normal', name='branch1_class')(x)
branch2_class = Dense(1, name='branch2_class')(x)
out = keras.layers.concatenate([branch1_class, branch2_class], name='concat')
model = keras.models.Model(input, out)
model.compile(loss=custom_loss2, optimizer='adam',  metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 21)]         0           []                               
                                                                                                  
 dense_1 (Dense)                (None, 32)           704         ['input[0][0]']                  
                                                                                                  
 dropout_1 (Dropout)            (None, 32)           0           ['dense_1[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 64)           2112        ['dropout_1[0][0]']              
                                                                                              

In [12]:
hist = model.fit(X_train, y_train, validation_data=[X_valid, y_valid], epochs=20)

Epoch 1/20
2104/2104 [==============================] - 42s 17ms/step - loss: 5612504.5000 - accuracy: 0.5045 - val_loss: 329.2167 - val_accuracy: 0.0107
Epoch 2/20
2104/2104 [==============================] - 39s 19ms/step - loss: 150606.9219 - accuracy: 0.5090 - val_loss: 21430.6680 - val_accuracy: 0.9893
Epoch 3/20
2104/2104 [==============================] - 42s 20ms/step - loss: 50306.0469 - accuracy: 0.5126 - val_loss: 21.6205 - val_accuracy: 0.9893
Epoch 4/20
2104/2104 [==============================] - 41s 19ms/step - loss: 15060.1250 - accuracy: 0.5189 - val_loss: 4042.4966 - val_accuracy: 0.0107
Epoch 5/20
2104/2104 [==============================] - 38s 18ms/step - loss: 3793.5408 - accuracy: 0.5505 - val_loss: 468.4433 - val_accuracy: 0.0107
Epoch 6/20
2104/2104 [==============================] - 37s 17ms/step - loss: 566.0189 - accuracy: 0.5750 - val_loss: 22.8934 - val_accuracy: 0.9893
Epoch 7/20
2104/2104 [==============================] - 36s 17ms/step - loss: 127.9247 

KeyboardInterrupt: 

In [101]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


AttributeError: 'NoneType' object has no attribute 'create'

In [ ]:
y_pred = model.predict(X_test)

260/260 [==============================] - 1s 3ms/step


In [ ]:
d = pd.DataFrame(y_pred)
d[0] = d[d[0] <= 0.5][0] = 0

In [ ]:
s = d.to_numpy()

In [107]:
accuracy = accuracy_score(pd.DataFrame(y_test)[0], pd.DataFrame(s)[0])

In [ ]:
confusion_matrix(pd.DataFrame(y_test)[0], pd.DataFrame(s)[0])

In [108]:
print('\n정확도: {:.4f}'.format(accuracy))


정확도: 0.4977


In [86]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score, confusion_matrix

def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))    
get_clf_eval(y_test, s)

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and continuous-multioutput targets

In [19]:
# for i in epoch:
#     for j in batch_sizes:
#         model_file_name = "mt_model_"+str(i)+"_"+str(j)
#         print("Model: "+model_file_name)
#         print("Epoch: "+str(i)+", Batch size: "+str(j))

#         model = Sequential()
#         model.add(Dense(128, input_shape=(data_x.shape[1],), activation='relu', kernel_initializer='normal'))
#         model.add(Dropout(rate=0.1))
#         model.add(Dense(128, input_shape=(data_x.shape[1],), activation='relu', kernel_initializer='normal'))
#         model.add(Dense(128, activation='relu', kernel_initializer='normal'))
#         model.add(Dense(1, activation='sigmoid', kernel_initializer='normal'))
#         model.add(Dense(1, activation='softmax', kernel_initializer='normal'))

#         model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])

#         model.fit(data_x, data_y, epochs=i, batch_size=j, validation_split=0.2, verbose=2)

#         y = model.predict(data_x)

#         model.save(model_file_name)
#         print()

Model: mt_model_32_8
Epoch: 32, Batch size: 8


ValueError: Unknown activation function: cross_entopy. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.